# Introduction to PyTerrier

_IN4325: Information retrieval lecture, TU Delft_

**Part 7: Neural ranking models**

In this part, we'll learn how to use **neural ranking models** with PyTerrier. Since PyTerrier itself does not implement any neural rankers, we'll use the [**Fast-Forward indexes** library](https://github.com/mrjleo/fast-forward-indexes), which focuses on efficient re-ranking using neural models and provides PyTerrier integration (using transformers). In order to learn more about Fast-Forward indexes, have a look at the [documentation](https://mrjleo.github.io/fast-forward-indexes/docs/v0.2.0/) and the [corresponding paper](https://dl.acm.org/doi/abs/10.1145/3485447.3511955).


In [1]:
pip install python-terrier==0.10.0 fast-forward-indexes==0.2.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


**Important**: In order to execute this notebook in reasonable time, you'll need a CUDA-capable GPU. If you have one, follow the [official tutorials](https://pytorch.org/get-started/locally/) and install PyTorch with CUDA acceleration. If you do not have one, you can use Google Colab: Under "Edit" -> "Notebook settings" -> "Hardware accelerator", select a GPU.

If the installation was successful, the following should return `True`:


In [1]:
import torch

torch.cuda.is_available()

True

_Side note_: In this notebook, we focus on the **retrieve-and-re-rank** setting. PyTerrier supports **dense retrieval** models through plugins (such as [pyterrier_ance](https://github.com/terrierteam/pyterrier_ance)). Since dense retrieval is often very resource-demanding, we do not cover it here. Another library that provides many pre-trained models and dense retrieval indexes is [pyserini](https://github.com/castorini/pyserini).


In [2]:
import pyterrier as pt

if not pt.started():

    pt.init(tqdm="notebook")

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In this notebook, we'll use the [FiQA dataset](https://sites.google.com/view/fiqa/home), which is a dataset for financial question answering. Since QA pipelines usually include a retrieval/ranking step, FiQA provides a corpus, queries (questions), and corresponding QRels:


In [3]:
dataset = pt.get_dataset("irds:beir/fiqa")

Due to the domain of this dataset (finance), the questions and documents are relatively complex. We'll see how much a large neural re-ranking model (based on BERT) manages to improve over term matching (BM25).

Let's create a lexical index first:


In [4]:
from pathlib import Path

indexer = pt.IterDictIndexer(
    str(Path.cwd()),  # this will be ignored
    type=pt.index.IndexingType.MEMORY,
)
index_ref = indexer.index(dataset.get_corpus_iter(), fields=["text"])

beir/fiqa documents:   0%|          | 0/57638 [00:00<?, ?it/s]

As our baseline, we measure BM25 performance without any re-ranking:


In [5]:
from pyterrier.measures import RR, nDCG, MAP

bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")
testset = pt.get_dataset("irds:beir/fiqa/test")
pt.Experiment(
    [~bm25],
    testset.get_topics(),
    testset.get_qrels(),
    eval_metrics=[RR @ 10, nDCG @ 10, MAP @ 100],
)

,name,RR@10,nDCG@10,AP@100
0,Cache(BR(BM25)),0.310271,0.252589,0.20864


## Fast-Forward indexes

Fast-forward indexes use _dual-encoder models_ (the same that are used in dense retrieval) for _interpolation-based re-ranking_. The benefit of this (compared to cross-encoders) is that document representations only need to be computed once (during the indexing step) and can be looked up during re-ranking.

### The encoders

We'll start by instantiating the encoders. [TCT-ColBERT](https://github.com/castorini/tct_colbert) is a single-vector dual-encoder model based on BERT, where the query and document encoders are identical (Siamese architecure). A pre-trained model (trained on MS MARCO) is [available on the Hugging Face hub](https://huggingface.co/castorini/tct_colbert-msmarco). We'll use this model in a transfer setting (i.e., without fine-tuning) on the FiQA dataset.

The encoders can be loaded as follows:


In [6]:
from fast_forward.encoder import TCTColBERTQueryEncoder, TCTColBERTDocumentEncoder
import torch

q_encoder = TCTColBERTQueryEncoder("castorini/tct_colbert-msmarco")
d_encoder = TCTColBERTDocumentEncoder(
    "castorini/tct_colbert-msmarco",
    device="cuda:0" if torch.cuda.is_available() else "cpu",
)

Now we can use these encoders to compute vector representations of text pieces, such as queries:


In [7]:
q_encoder(["query 1", "query 2"])

array([[-0.105638  ,  0.02958424,  0.13244788, ...,  0.02808921,
        -0.00611512,  0.0177914 ],
       [-0.16003011,  0.02995185,  0.19217151, ..., -0.02004313,
        -0.02355196, -0.019374  ]], dtype=float32)

### The index

We've already created an index for the BM25 retriever earlier. For the dense vector representations, we'll need to create another separate index:


In [8]:
from fast_forward import OnDiskIndex, Mode

ff_index = OnDiskIndex(
    Path("ffindex_fiqa_tct.h5"), dim=768, query_encoder=q_encoder, mode=Mode.MAXP
)

ValueError: File ffindex_fiqa_tct.h5 exists.

We set `dim=768`, because our encoders output `768`-dimensional representations. `Mode.MAXP` determines how documents that have multiple vectors are scored; however, since our documents only have a single representation each, this will have no effect; we just need to set this to tell the index we're working with documents and not passages (more on that [here](https://mrjleo.github.io/fast-forward-indexes/docs/v0.2.0/fast_forward/index.html#ranking-mode)).

Now we can index the corpus using our document encoder. This is done using the `Indexer` utility class. We'll use an iterator that simply yields the documents in the correct format.

**The next cell will take a while to execute, even with GPU acceleration.** You can adjust the batch size according to your available VRAM.


In [ ]:
from fast_forward import Indexer


def docs_iter():
    for d in dataset.get_corpus_iter():
        yield {"doc_id": d["docno"], "text": d["text"]}


ff_indexer = Indexer(ff_index, d_encoder, batch_size=8)
ff_indexer.index_dicts(docs_iter())







Once indexing is done, we can always load the index instead of indexing everything again:


In [12]:
from fast_forward import OnDiskIndex, Mode

ff_index = OnDiskIndex.load(
    Path("ffindex_fiqa_tct.h5"), query_encoder=q_encoder, mode=Mode.MAXP
)

100%|██████████| 57638/57638 [00:00<00:00, 386564.57it/s]


At this point, if you have enough RAM, you can load the entire index (i.e., all vector representations) into the main memory:


In [13]:
ff_index = ff_index.to_memory()

## Re-ranking BM25 results

We've already performed re-ranking in the learning-to-rank notebook. In order to use a Fast-Forward index for re-ranking, we wrap it in an `FFScore` transformer:


In [14]:
from fast_forward.util.pyterrier import FFScore

ff_score = FFScore(ff_index)

In order to see how this works, we take the queries from the testset and retrieve a small number of candidate documents for each one using BM25:


In [15]:
candidates = (~bm25 % 5)(testset.get_topics())
candidates

,qid,docid,docno,rank,score,query
0,4641,36224,376148,0,41.677305,where should i park my rainy day emergency fund
1,4641,47916,497993,1,29.149791,where should i park my rainy day emergency fund
2,4641,55690,580025,2,26.773005,where should i park my rainy day emergency fund
3,4641,24501,253614,3,26.640181,where should i park my rainy day emergency fund
4,4641,3157,32833,4,24.265187,where should i park my rainy day emergency fund
...,...,...,...,...,...,...
644153,2399,33136,343489,0,33.280064,where do web sites get foreign exchange curren...
644154,2399,6704,69171,1,31.335596,where do web sites get foreign exchange curren...
644155,2399,4148,43046,2,31.265964,where do web sites get foreign exchange curren...
644156,2399,46670,484891,3,29.869008,where do web sites get foreign exchange curren...


As usual, this data frame contains a score for each query-document pair. We can apply our new re-ranking transformer to the candidates:


In [16]:
re_ranked = ff_score(candidates)
re_ranked

,qid,docno,score_0,score,query
0,4641,376148,41.677305,67.637367,where should i park my rainy day emergency fund
1,4641,497993,29.149791,68.335365,where should i park my rainy day emergency fund
2,4641,580025,26.773005,67.524490,where should i park my rainy day emergency fund
3,4641,253614,26.640181,67.098091,where should i park my rainy day emergency fund
4,4641,32833,24.265187,68.345680,where should i park my rainy day emergency fund
...,...,...,...,...,...
3235,2399,343489,33.280064,69.723114,where do web sites get foreign exchange curren...
3236,2399,69171,31.335596,67.643097,where do web sites get foreign exchange curren...
3237,2399,43046,31.265964,67.293716,where do web sites get foreign exchange curren...
3238,2399,484891,29.869008,68.099777,where do web sites get foreign exchange curren...


As you can see, the `score` column has now been updated to reflect the re-ranking scores. Furthermore, there is a new column, `score_0`, which contains the original retrieval scores. As mentioned earlier, Fast-Forward indexes focus on _interpolation-based re-ranking_. In essence, the idea is to take both lexical retrieval scores $s_{\text{lex}}$ and semantic re-ranking scores $s_{\text{sem}}$ into account, such that the final score $s$ is computed as follows:

$$s = \alpha s_{\text{lex}} + (1-\alpha) s_{\text{sem}}$$

We can perform the interpolation using the `FFInterpolate` transformer:


In [17]:
from fast_forward.util.pyterrier import FFInterpolate

ff_int = FFInterpolate(alpha=0.5)
ff_int(re_ranked)

,qid,docno,query,score
0,4641,376148,where should i park my rainy day emergency fund,54.657336
1,4641,497993,where should i park my rainy day emergency fund,48.742578
2,4641,580025,where should i park my rainy day emergency fund,47.148748
3,4641,253614,where should i park my rainy day emergency fund,46.869136
4,4641,32833,where should i park my rainy day emergency fund,46.305434
...,...,...,...,...
3235,2399,343489,where do web sites get foreign exchange curren...,51.501589
3236,2399,69171,where do web sites get foreign exchange curren...,49.489347
3237,2399,43046,where do web sites get foreign exchange curren...,49.279840
3238,2399,484891,where do web sites get foreign exchange curren...,48.984392


In the data frame above, both scores have been fused into one with equal weights.

Now we're ready to take everything for a spin. Let's compare our re-ranker to BM25:


In [18]:
pt.Experiment(
    [~bm25, ~bm25 % 100 >> ff_score >> ff_int],
    testset.get_topics(),
    testset.get_qrels(),
    eval_metrics=[RR @ 10, nDCG @ 10, MAP @ 100],
    names=["BM25", "BM25 >> FF"],
)

,name,RR@10,nDCG@10,AP@100
0,BM25,0.310271,0.252589,0.208640
1,BM25 >> FF,0.334666,0.274198,0.224551


As you can see, the performance already improved quite nicely. But thinking back, we chose $\alpha=0.5$ pretty much arbitrarily. How do we know this is really the best value?

### Validation

PyTerrier offers several functions to determine the best hyperparameters for a ranker. In the following, we'll use [`pyterrier.GridSearch`](https://pyterrier.readthedocs.io/en/latest/tuning.html#pyterrier.GridSearch) to find the best value for $\alpha$.

**Important**: When you tune hyperparameters of your model, **do not use the same data you use for testing (i.e., the testset)**. Otherwise, your results are invalid, because you optimized your method for the testing data. Instead, we'll use the development (validation) data:


In [ ]:
devset = pt.get_dataset("irds:beir/fiqa/dev")

PyTerriers `GridSearch` class can be used to automatically run an experiment multiple times in order to find the hyperparameters that result in the best performance.

We'll use a similar pipeline as before, but we limit the number of candidate documents to `100` in order to reduce the runtime. We provide a list of values for `alpha` and a metric (MAP), which is used to decide which value results in the best performance:


In [ ]:
pt.GridSearch(
    ~bm25 % 100 >> ff_score >> ff_int,
    {ff_int: {"alpha": [0.05, 0.1, 0.5, 0.9]}},
    devset.get_topics(),
    devset.get_qrels(),
    "map",
    verbose=True,
)

In this case, out of the options we provided, `alpha=0.1` was the best.

_Side note_: As of now, PyTerrier does not support caching for re-ranking transformers. Hence, `GridSearch` takes a long time, because the scores are re-computed every time, even though that wouldn't be necessary.

Conveniently, the best value has already been set for us in the transformer:


In [ ]:
ff_int.alpha

Let's repeat our experiment on the testset with the optimal hyperparameter:


In [ ]:
pt.Experiment(
    [~bm25, ~bm25 % 1000 >> ff_score >> ff_int],
    testset.get_topics(),
    testset.get_qrels(),
    eval_metrics=[RR @ 10, nDCG @ 10, MAP @ 100],
    names=["BM25", "BM25 >> FF"],
)

As you can see, the value of `alpha` makes a big difference!

_Final remarks_: We've used neural models for re-ranking only in this notebook. However, in practice, it is not uncommon to use the re-ranking scores we computed here as a feature for a learning-to-rank model.

## Further reading

Check out the [section on neural models](https://pyterrier.readthedocs.io/en/latest/neural.html) in the PyTerrier documentation.
